# Incompressible Navier Stokes Solver

# Index
1. [Introduction](#Introduction)
2. [Governing Equations](#Governing-Equations)
    - [Continuity equation](#Continuity-equation)
    - [Momentum equations](#Momentum-equations)
    - [Challenges in Solving the Incompressible Navier-Stokes Equations](#Challenges-in-Solving-the-Incompressible-Navier-Stokes-Equations)
4. [Discretization](#Discretization)
5. [Grid Types](#Grid-Types)
6. [Algorithm](#Algorithm)
7. [Poisson solver](#Poisson-solver)
   - [Jacobi](#Jacobi)
   - [Gauss-Seidel](#Gauss-Seidel)
8. [Results](#results)
9. [Conclusion](#conclusion)

# Introduction
Fluid dynamics plays a vital role in understanding natural phenomena and designing engineering systems. From the flow of air around an airplane wing to the movement of water in rivers, the behavior of fluids is governed by a set of partial differential equations known as the Navier-Stokes equations.

This notebook focuses on the design of a computational method to solve the 2D incompressible Navier-Stokes equations, which describe the motion of fluids with constant density. These equations are widely applicable in scenarios where density variations are negligible, such as liquid flows or low-speed gas flows.

This notebook introduces the incompressible Navier-Stokes equations, examines the challenges involved in solving them, and presents a numerical solver designed to address these challenges efficiently, comparing different types of mesh, interpolation and iterative solvers. 

# Governing Equations
The incompressible Navier-Stokes equations describe the motion of fluid flows where the density remains constant. These equations are fundamental in fluid dynamics and are used to model a wide variety of physical phenomena, such as airflow over airplane wings or water flow through pipes. The original motivation to develop this solver was to model the wake formation of wind turbines in a wind farm.

The equations consist of two main components:

* **Continuity Equation:** Ensures the conservation of mass in incompressible flows.
* **Momentum Equations:** Represent the conservation of momentum and describe how the velocity field evolves over time due to various forces.
## Continuity equation
The continuity equation ensures the conservation of mass in incompressible flows. For a velocity field 
u, the equation is written as $$∇⋅u= 0$$ In 2D derivative form, it becomes:  $$\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} = 0$$
## Momentum equations
The momentum equations describe the conservation of momentum in the fluid. For incompressible flows, they are expressed as:
$$\frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla)\mathbf{u} = -\frac{1}{\rho} \nabla p + \nu \nabla^2 \mathbf{u} + \mathbf{f}$$
2D derivative form:

**x-Momentum:**
$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} = -\frac{1}{\rho} \frac{\partial p}{\partial x} + \nu \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right)$$
**y-Momentum:**
$$\frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} = -\frac{1}{\rho} \frac{\partial p}{\partial y} + \nu \left( \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} \right)$$
## EXPLAIN EACH OF THE TERMS ????




## Challenges in Solving the Incompressible Navier-Stokes Equations

### Coupling Between Velocity and Pressure
A key challenge in solving the incompressible Navier-Stokes equations is the coupling between the velocity u and the pressure p. In this version of the equations, they appear to be decoupled, however they must be copied because: 
* The velocity field influences the pressure distribution.
* The pressure must ensure that the velocity field satisfies the incompressibility constraint $$
\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} = 0
$$
This coupling leads to:

* No explicit equation for pressure: The pressure field must be computed indirectly through the velocity field.
* Pressure-velocity decoupling issues: Without careful numerical treatment, unphysical solutions may arise.

Given the lack of a direct link for pressure in between continuity and momentum equations, an additional Poisson equation can be derived to substitute the continuity equation. 
## Explain the derivation of that equation HERE (check my Ipad and LB notes). 

### Nonlinearity of the Momentum Equations
The advection term $(\mathbf{u} \cdot \nabla)\mathbf{u}$ in the momentum equations introduces nonlinearity, leading to:
* Instabilities: Numerical errors may grow exponentially without stabilization techniques.
* High computational cost: Iterative solvers are often required for convergence.
### Boundary Conditions
Properly specifying boundary conditions is critical but can be challenging. Common issues include:
* Matching inflow and outflow conditions.
* Satisfying the no-slip condition at solid walls.
* Divergence close to the boundaries. 
  
### Discretization Challenges
Numerical solvers rely on discretizing the equations, which introduces its own set of challenges:
* Accuracy vs. computational cost: High-resolution grids improve accuracy but increase computational demands.
* Numerical diffusion: Excessive artificial smoothing of the solution can obscure key flow features.
* Pressure-velocity consistency: Ensuring that the discrete velocity field satisfies continuity.


# Discretization 
EXPLAIN how the equations were discretized on the staggered (and collocated) grid, including the different used interpolations. 

# Grid Types 
Numerical solvers for the incompressible Navier-Stokes equations often use structured grids to discretize the computational domain. Two common types of grids are **collocated grids** and **staggered grids**. This section explains these grid configurations, discusses why staggered grids are often preferred for maintaining incompressibility and compares the results of using the different grids for the same cases. 
## Collocated Grid 
ADD EXPLANATION AND PICTURE HERE (check my notes). 
## Staggered Grid
ADD EXPLANATION AND PICTURE HERE (check my notes). Advocate potential advantages 
## Comparison and table of advantages and disadvantages? 




# Algorithm
In this section, we outline and compare two different numerical algorithms employed to solve the incompressible Navier-Stokes equations: **Chorin's Projection Method** and the **Predictor-Corrector Method**. These algorithms differ in their sequence of operations and the way they integrate the pressure and velocity fields over time. I will explain and implement both methods and compare their performance and results in terms of accuracy and computational efficiency.
## 1st order unsplit Euler's method (cite Laurena Barba's method)  
Chorin's projection method is a widely used technique to solve the incompressible Navier-Stokes equations. The primary goal of this method is to ensure that the computed velocity field remains divergence-free (i.e., mass-conserving) at each timestep. This is achieved through two main steps: pressure projection and advection-diffusion, which are alternated to ensure the incompressibility condition.
### Step 1. Solving the Poisson equation for pressure correction.
The first step in Chorin’s method is to solve the Poisson equation for the pressure, which is derived from the incompressibility condition $$\nabla^2 p = \nabla \cdot \mathbf{u}^*$$ The ultimate goal is to calculate the pressure gradient that will ensure zero divergence when performing the advection-diffusion step. 
### Step 2. Advection-Diffusion with Pressure Gradient.
The advection-diffusion step updates the velocity field by incorporating the pressure gradient obtained from the Poisson solver (Step 1). This step ensures the incompressibility of the flow by adjusting the velocity field based on the pressure distribution, while simultaneously advecting and diffusing the fluid. The advection-diffusion equation, which includes the pressure gradient, is:
$$
\frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u} = -\frac{1}{\rho} \nabla p + \nu \nabla^2 \mathbf{u}
$$

## Chorin's Projection method 
The predictor-corrector algorithm is a widely used method to solve fluid dynamics equations, particularly when dealing with incompressible flows. 
The idea is to perform the advection-diffusion step without considering the pressure gradient. This step gives us an intermediate velocity field that may not be divergence-free (it does not satisfy the incompressibility condition). Then, we correct this predicted velocity field to ensure incompressibility, using the pressure gradient computed from the Poisson equation.

### Prediction Step (Advection-Diffusion).
In the prediction step, the velocity field is updated by solving the advection-diffusion equation without considering the pressure gradient term. This means that the velocity field evolves based on the advection of the fluid and the diffusion effects, but the incompressibility constraint is not enforced at this stage.
$$
\frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u} = \nu \nabla^2 \mathbf{u}
$$
The result of the prediction step is an intermediate velocity field which is used in the next step.

### Correction Step (Apply Pressure Gradient term)
Once we have the intermediate velocity field, we apply the Poisson equation to solve for the pressure field. The Poisson equation ensures that the velocity field satisfies the incompressibility condition ∇⋅u=0. The equation for pressure p is:
$$
\nabla^2 p = \nabla \cdot \mathbf{u}^*
$$
After solving for the pressure p, we compute the pressure gradient and this gradient is then used to correct the velocity field. The corrected velocity field is computed by subtracting the pressure gradient term from the intermediate velocity:
### Correct this (flip rho grad p
$$
\mathbf{u} = \mathbf{u}^* - \frac{\rho \Delta t}{\nabla p}
$$
This method really ensures zero divergence because the Poisson equation is solved for the actual velocity field. 



# Poisson solver 
In incompressible flows, ensuring mass conservation requires solving the Poisson equation for pressure at each time step, based on the updated velocity field. This step is crucial for projecting the velocity field to satisfy the continuity equation:
where $$\nabla^2 p = \nabla \cdot \mathbf{u}^*$$
where p is the intermediate velocity field computed from the momentum equation.
To solve this equation efficiently, two common iterative solvers are used: Jacobi and Gauss-Seidel methods. In this section, I will explain both methods step by step, provide scripts for their implementation, and compare their performance.

## Jacobi 
The Jacobi method computes the solution iteratively by solving for each variable in terms of the others using values from the previous iteration.
### Algorithm steps 
1. **Initialize variables.** Start with the current p field & define the RHS (b) of the Poisson equation (derived from velocity divergence). 
2. **Precompute coefficients.** Precompute p_coef and b, which are adjusted for the grid spacings. $$p_{\text{coef}} = \frac{1}{2(\Delta x^2 + \Delta y^2)}$$ $$b_{i,j} \leftarrow b_{i,j} \cdot \frac{2(\Delta x^2 + \Delta y^2) \rho}{\Delta x^2 \Delta y^2}$$ The computation of b depends on the method used (projection or predictor-corrector). 
4. **Iteration.** Jacobian update of p on the interior grid points.  $$p_{i,j}^{(k+1)} = p_{\text{coef}} \left[ (p_{i+1,j}^{(k)} + p_{i-1,j}^{(k)}) \Delta y^2 + (p_{i,j+1}^{(k)} + p_{i,j-1}^{(k)}) \Delta x^2 \right] - b_{i,j}$$
5. **Enforce Boundary Conditions** Apply Neumann boundary conditions $\frac{\partial p}{\partial n} = 0$ in this case. This may change depending on the BC problem.
6. **Compute Error** Calculate the root-mean-square (RMS) error between successive pressure fields: $$\text{Error} = \sqrt{\frac{1}{N} \sum_{i,j} \left( p_{i,j}^{(k+1)} - p_{i,j}^{(k)} \right)^2}$$
7. **End of the iteration** Iteration automatically ends if: 
    A) Error is lower than tolerance.
    B) Maximum number of iterations is reached.
8. **Output** Return the final pressure field, which satisfies the Poisson equation within the specified tolerance.

In [ ]:
def pressure_poisson(p, b, dx, dy, tol, maxiter):
    """
    Solve the Poisson equation for pressure correction using Jacobi's iterative method.

    Parameters:
    -----------
    p : numpy.ndarray
        Current pressure field. This array will be updated iteratively.
    b : numpy.ndarray
        Right-hand side of the Poisson equation, derived from velocity divergence.
    dx, dy : float
        Grid spacing in the x and y directions.
    tol : float
        Convergence tolerance for the root-mean-square error.
    maxiter : int
        Maximum number of iterations. Accelerates the speed at the beginning of the iterations. 
    rho : density. 

    Returns:
    --------
    numpy.ndarray
        The updated pressure field that satisfies the Poisson eq. within the specified tolerance.

    Notes:
    ------
    - Implements Jacobi's method, iteratively updating the pressure field.
    - Enforces Neumann boundary conditions (zero pressure gradient) on all domain edges (this is just for the Cavity Flow case). 
    - The method stops when either the error falls below the specified tolerance or the maximum
      number of iterations is reached.
    """
    err = np.inf # Initialize huge error.
    nit = 0 # Reset num iterations.
    pcoef = 0.5 / (dx**2 + dy**2) # Simplifies code
    b *= rho * dx**2 * dy**2 / (2*(dx**2 + dy**2))

    while err > tol and nit < maxiter:
        pn = p.copy()

        p[1:-1, 1:-1] = (pcoef * ((pn[1:-1, 2:] + pn[1:-1, :-2])*dy**2
                        + (pn[2:, 1:-1] + pn[:-2, 1:-1])*dx**2) - b)

        # BCs. Openfield.
        p[:, 0] = p[:, 1] # dp/dx=0 at x=0.
        p[:, -1] = -p[:, -2] # p = 0 at x = L.
        p[0, :] = p[1, :]   # dp/dy = 0 at y = 0.
        p[-1, :] = p[-2, :] # dp/dx = 0 at y = 2.

        err = np.mean((p[1:-1, 1:-1] - pn[1:-1, 1:-1])**2)**0.5
        nit += 1

    return p

## Gauss-Seidel
The Gauss-Seidel method improves on Jacobi's iterative solver by updating the pressure values in-place, making it more computationally efficient. What is more, it has been implemented using Cython for an even faster convergence.
### Algorithm steps 
1. **Initialize variables.** Start with the current p field & define the RHS (b) of the Poisson equation (derived from velocity divergence). 
2. **Precompute coefficients.** Precompute p_coef and b, which are adjusted for the grid spacings. $$p_{\text{coef}} = \frac{1}{2(\Delta x^2 + \Delta y^2)}$$ $$b_{i,j} \leftarrow b_{i,j} \cdot \frac{2(\Delta x^2 + \Delta y^2) \rho}{\Delta x^2 \Delta y^2}$$ 
4. **Iteration.** Loop through the grid, updating the pressure values in-place at each grid point using the formula: $$p_{i,j} = p_{\text{coef}} \left[ (p_{i,j+1} + p_{i,j-1}) \Delta y^2 + (p_{i+1,j} + p_{i-1,j}) \Delta x^2 \right] - b_{i,j}$$
5. **Enforce Boundary Conditions** Apply Neumann boundary conditions $\frac{\partial p}{\partial n} = 0$ in this case. This may change depending on the BC problem.
6. **Compute Error** Calculate the root-mean-square (RMS) error between successive pressure fields: $$\text{Error} = \sqrt{\frac{1}{N} \sum_{i,j} \left( p_{i,j}^{(k+1)} - p_{i,j}^{(k)} \right)^2}$$
7. **End of the iteration** Iteration automatically ends if: 
    A) Error is lower than tolerance.
    B) Maximum number of iterations is reached.
8. **Output** Return the final pressure field, which satisfies the Poisson equation within the specified tolerance.


In [ ]:
def pressure_poisson_gauss_seidel(p, b, dx, dy, rho):
    """
    Solve the Poisson equation for pressure correction using the Gauss-Seidel method.

    This function iteratively solves the pressure Poisson equation, which is derived from 
    the incompressible Navier-Stokes equations to ensure mass conservation. It uses the 
    Gauss-Seidel method for in-place updates, leveraging the latest pressure estimates 
    during each iteration for faster convergence.

    Parameters:
    -----------
    p : numpy.ndarray
        The pressure field (2D array) that needs to be updated in order to satisfy the Poisson equation. 
    b : numpy.ndarray
        The Poisson's equation RHS (b term, 2D array) derived from the velocity divergence.
    dx : float
        Grid spacing in the x-direction.
    dy : float
        Grid spacing in the y-direction.
    rho : float
        Fluid density, used to scale the source term.

    Returns:
    --------
    p : numpy.ndarray
        Updated pressure field satisfying the Poisson equation within the specified tolerance.

    Key Features:
    --------------
    1. In-place updates using Gauss-Seidel accelerate convergence compared to Jacobi's method.
    2. Enforces Neumann boundary conditions (zero pressure gradient) on all domain edges (this is just for the Cavity Flow case).
    3. Convergence is determined based on the root-mean-square (RMS) error between iterations.
    """
    err = np.inf  # Initialize a large error.
    nit = 0  # Reset the number of iterations.
    pcoef = 0.5 / (dx**2 + dy**2)  # Precompute coefficient for simplicity.
    b *= rho * dx**2 * dy**2 / (2 * (dx**2 + dy**2))

    while err > tol and nit < maxiter:
        pn = p.copy()

        # Gauss-Seidel in-place update
        p = gauss_seidel_iteration(p, b, pcoef, dx, dy)

        # Apply boundary conditions
        p[:, 0] = p[:, 1] # dp/dx=0 at x=0.
        p[:, -1] = -p[:, -2] # p = 0 at x = L.
        p[0, :] = p[1, :]   # dp/dy = 0 at y = 0.
        p[-1, :] = p[-2, :] # dp/dx = 0 at y = 2.

        # Calculate error based on the new values
        err = np.mean((p[1:-1, 1:-1] - pn[1:-1, 1:-1])**2)**0.5
        nit += 1

    return p

### Performance Optimization with Cython
To improve the performance of the iterative solver, Cython is used. Cython is a superset of Python that allows for the inclusion of C-like performance optimizations while maintaining the ease of Python syntax. By compiling the Python code into C, it allows the iterative solver to execute much faster, which is essential for large grid sizes in computational fluid dynamics simulations.
Specifically, the gauss_seidel_iteration function is implemented using Cython's typed memoryviews (cnp.ndarray) to directly interact with NumPy arrays and avoid the overhead of Python's dynamic typing. This allows for efficient manipulation of large datasets typical in fluid simulations.

In [ ]:
def gauss_seidel_iteration(cnp.ndarray[cnp.double_t, ndim=2] p,
                           cnp.ndarray[cnp.double_t, ndim=2] b,
                           double pcoef,
                           double dy,
                           double dx):
    cdef int i,j

    for i in range(1,p.shape[0]-1):
        for j in range(1,p.shape[1]-1):
            p[i,j] = pcoef * ((p[i,j+1] + p[i,j-1]) * dy**2
                                + (p[i+1,j] + p[i-1,j]) * dx**2) - b[i-1,j-1]

    return p

Instructions for usage:
1. Recompile the pyx file: python setup.py build_ext --inplace
2. Run main.py normally 

References: 
- Cython: https://cython.org/
- Gauss Seidel and Jacobi. 
- Incompressible NS (Kelsea Boom)
- Lorena Barba
- Chorins paper
- Peter's 3030 course TextBook (Check which one he s using). 